<a href="https://colab.research.google.com/github/VictorOPreuss/MeLi-Data-Challenge-2020/blob/main/MeLi_Data_Challenge_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bibliotecas**

In [ ]:
!sudo apt-get update

In [2]:

# Install spark-related dependencies
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.0.1-bin-hadoop2.7.tgz

!pip install -q findspark
!pip install pyspark
# Set up required environment variables

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"


     |████████████████████████████████| 204.2MB 61kB/s 
     |████████████████████████████████| 204kB 41.6MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=141fa6b45951041cc87a3c503d7ce2c82faccfecb8d12fe42323139b999f6faf
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [3]:
# Tools we need to connect to the Spark server, load our data,
# clean it and prepare it
from pyspark import SparkContext
from pyspark.sql import SparkSession

from pyspark.sql.functions import *

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import os
import requests 
import zipfile
from google.colab import files

# settings
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 200)

In [4]:
import findspark
findspark.init("spark-3.0.1-bin-hadoop2.7")# SPARK_HOME
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

# **ETL**

In [5]:

! wget https://meli-data-challenge.s3.amazonaws.com/2020/train_dataset.jl.gz
! wget https://meli-data-challenge.s3.amazonaws.com/2020/test_dataset.jl.gz
! wget https://meli-data-challenge.s3.amazonaws.com/2020/item_data.jl.gz
! wget https://meli-data-challenge.s3.amazonaws.com/2020/sample_submission.csv


--2020-11-14 20:09:00--  https://meli-data-challenge.s3.amazonaws.com/2020/train_dataset.jl.gz
Resolving meli-data-challenge.s3.amazonaws.com (meli-data-challenge.s3.amazonaws.com)... 52.217.9.204
Connecting to meli-data-challenge.s3.amazonaws.com (meli-data-challenge.s3.amazonaws.com)|52.217.9.204|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 128022297 (122M) [application/x-gzip]
Saving to: ‘train_dataset.jl.gz’

train_dataset.jl.gz 100%[===================>] 122.09M  87.3MB/s    in 1.4s    

2020-11-14 20:09:02 (87.3 MB/s) - ‘train_dataset.jl.gz’ saved [128022297/128022297]

--2020-11-14 20:09:02--  https://meli-data-challenge.s3.amazonaws.com/2020/test_dataset.jl.gz
Resolving meli-data-challenge.s3.amazonaws.com (meli-data-challenge.s3.amazonaws.com)... 52.217.9.204
Connecting to meli-data-challenge.s3.amazonaws.com (meli-data-challenge.s3.amazonaws.com)|52.217.9.204|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 54305870 (52M) 

In [6]:

! gunzip train_dataset.jl.gz
! gunzip test_dataset.jl.gz
! gunzip item_data.jl.gz


In [7]:
train_path = '/content/train_dataset.jl'
test_path = '/content/test_dataset.jl'
item_data_path = '/content/item_data.jl'

In [8]:
train = spark.read.json(train_path, multiLine= False)
test = spark.read.json(test_path, multiLine= False)
item_data = spark.read.json(item_data_path)
sample_submission = pd.read_csv('sample_submission.csv')

In [9]:
train.printSchema()

root
 |-- item_bought: long (nullable = true)
 |-- user_history: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- event_info: string (nullable = true)
 |    |    |-- event_timestamp: string (nullable = true)
 |    |    |-- event_type: string (nullable = true)



In [10]:
test.printSchema()

root
 |-- user_history: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- event_info: string (nullable = true)
 |    |    |-- event_timestamp: string (nullable = true)
 |    |    |-- event_type: string (nullable = true)



In [11]:
item_data.printSchema()

root
 |-- category_id: string (nullable = true)
 |-- condition: string (nullable = true)
 |-- domain_id: string (nullable = true)
 |-- item_id: long (nullable = true)
 |-- price: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- title: string (nullable = true)



In [12]:
train_index = train.select("*").withColumn("id", monotonically_increasing_id())
test_index = test.select("*").withColumn("id", monotonically_increasing_id())

In [13]:
train_final = (train_index
        .select("item_bought", "id", 
explode("user_history").alias("new_user_history"))
        .select("item_bought", "id", "new_user_history.*"))

In [14]:
test_final = (test_index
        .select("id", explode("user_history").alias("new_user_history"))
        .select("id", "new_user_history.*"))

# **EDA & Prep - Baseline**

In [16]:
train_final.show(10)

+-----------+---+------------------+--------------------+----------+
|item_bought| id|        event_info|     event_timestamp|event_type|
+-----------+---+------------------+--------------------+----------+
|    1748830|  0|           1786148|2019-10-19T11:25:...|      view|
|    1748830|  0|           1786148|2019-10-19T11:25:...|      view|
|    1748830|  0|RELOGIO SMARTWATCH|2019-10-19T11:26:...|    search|
|    1748830|  0|           1615991|2019-10-19T11:27:...|      view|
|    1748830|  0|           1615991|2019-10-19T11:28:...|      view|
|    1748830|  0|           1615991|2019-10-19T11:28:...|      view|
|    1748830|  0|           1615991|2019-10-19T11:30:...|      view|
|    1748830|  0|           1615991|2019-10-19T21:51:...|      view|
|    1748830|  0|           1615991|2019-10-19T21:52:...|      view|
|    1748830|  0|           1615991|2019-10-19T21:52:...|      view|
+-----------+---+------------------+--------------------+----------+
only showing top 10 rows



In [17]:
test_final.show(10)

+---+----------+--------------------+----------+
| id|event_info|     event_timestamp|event_type|
+---+----------+--------------------+----------+
|  0|   1572239|2019-09-26T18:31:...|      view|
|  0|   1572239|2019-09-26T18:35:...|      view|
|  0|   1572239|2019-09-26T18:37:...|      view|
|  0|   1572239|2019-09-26T18:38:...|      view|
|  0|   1572239|2019-09-26T18:40:...|      view|
|  0|   1572239|2019-09-26T18:40:...|      view|
|  0|   1572239|2019-09-26T18:41:...|      view|
|  0|   1572239|2019-09-26T18:41:...|      view|
|  0|   1572239|2019-09-26T19:03:...|      view|
|  0|   1194894|2019-09-27T21:33:...|      view|
+---+----------+--------------------+----------+
only showing top 10 rows



In [18]:
item_data.show(10)

+-----------+---------+--------------------+-------+----------+----------+--------------------+
|category_id|condition|           domain_id|item_id|     price|product_id|               title|
+-----------+---------+--------------------+-------+----------+----------+--------------------+
|  MLM170527|      new|MLM-INDIVIDUAL_HO...| 111260|1150000.00|      null|Casa Sola En Vent...|
|  MLM151595|      new|     MLM-VIDEO_GAMES| 871377|   1392.83|  15270800|Resident Evil Ori...|
|    MLM7697|      new|          MLM-SKIRTS| 490232|    350.00|      null|Falda De Imitació...|
|    MLM9761|     used|  MLM-GRAPHICS_CARDS|1150706|   3200.00|      null|Powercolor Red De...|
|    MLM1652|     used|       MLM-NOTEBOOKS| 934912|   1599.00|      null|Laptop Hp Nx6320 ...|
|   MLM92472|      new|MLM-VEHICLE_ACCES...| 534737|    470.00|      null|Transmisor Fm Sin...|
|  MLM167442|      new|MLM-CELLPHONE_COVERS| 369182|    589.00|      null|Funda Cartera Cas...|
|    MLM1588|      new|MLM-WALL_AND_CEIL

In [19]:
train_final.dtypes

[('item_bought', 'bigint'),
 ('id', 'bigint'),
 ('event_info', 'string'),
 ('event_timestamp', 'string'),
 ('event_type', 'string')]

In [20]:
item_data.dtypes

[('category_id', 'string'),
 ('condition', 'string'),
 ('domain_id', 'string'),
 ('item_id', 'bigint'),
 ('price', 'string'),
 ('product_id', 'string'),
 ('title', 'string')]

In [21]:
item_data.createGlobalTempView("metadata")
train_final.createGlobalTempView("train")
test_final.createGlobalTempView("test")

In [22]:
domain_query = spark.sql("""

SELECT 
  domain_id,
  COUNT(*) AS count,
  AVG(price) AS avg_price,
  MAX(price) AS max_price,
  MIN(price) AS min_price
FROM global_temp.metadata
GROUP BY domain_id
SORT BY count DESC

""").toPandas()

In [23]:
print(domain_query.shape)
domain_query.head()

(7894, 5)


,domain_id,count,avg_price,max_price,min_price
0,MLB-MICROPHONES,2448,1130.967663,999999.00,10.00
1,MLB-COMPLETE_SKATEBOARDS,339,427.985015,999.00,100.00
2,MLB-BICYCLE_AND_MOTORCYCLE_ALARMS,317,288.686530,99.90,100.21
3,MLB-AQUARIUM_FILTER_MATERIALS,305,120.349213,99.99,10.99
4,MLB-FLOOR_LAMPS,246,403.407073,99.98,100.00


In [24]:
category_query = spark.sql("""

SELECT 
  category_id,
  COUNT(*) AS count,
  AVG(price) AS avg_price,
  MAX(price) AS max_price,
  MIN(price) AS min_price
FROM global_temp.metadata
GROUP BY category_id
SORT BY count DESC

""").toPandas()

In [25]:
print(category_query.shape)
category_query.head()

(11493, 5)


,category_id,count,avg_price,max_price,min_price
0,MLB188065,10130,153.410706,9999.90,10.00
1,MLB189195,7431,332.518408,9999.00,10.00
2,MLB73055,1113,350.353522,99.99,100.00
3,MLB204802,1020,2047.404088,999999.00,10.00
4,MLB18395,888,107.421160,999.00,100.00


In [26]:
datetime_query = spark.sql("""

SELECT
  DATE_FORMAT(to_timestamp(event_timestamp, "yyyy-MM-dd'T'HH:mm:ss.SSS'-0400'"), "d M y") AS date,
  SUM(CASE WHEN event_type = 'view' then 1 else 0 end) AS sum_view,
  SUM(CASE WHEN event_type = 'search' then 1 else 0 end) AS sum_search
FROM global_temp.train
GROUP BY date
SORT BY sum_view DESC

""").toPandas()

In [27]:
print(datetime_query.shape)
datetime_query.head()

(38, 3)


,date,sum_view,sum_search
0,28 10 2019,71274,66958
1,27 9 2019,123211,127407
2,29 9 2019,207237,212330
3,31 10 2019,4123,3982
4,9 10 2019,209684,217686


In [28]:
train_query = spark.sql("""

SELECT 
  t.id,
  COUNT(DISTINCT t.event_info) AS unq_event_info,
  SUM(CASE WHEN t.event_type = 'view' then 1 else 0 end) AS sum_view,
  SUM(CASE WHEN t.event_type = 'search' then 1 else 0 end) AS sum_search,
  MIN(to_timestamp(t.event_timestamp, "yyyy-MM-dd'T'HH:mm:ss.SSS'-0400'")) AS first_timestamp,
  MAX(to_timestamp(t.event_timestamp, "yyyy-MM-dd'T'HH:mm:ss.SSS'-0400'")) AS last_timestamp,
  (UNIX_TIMESTAMP(MAX(to_timestamp(t.event_timestamp, "yyyy-MM-dd'T'HH:mm:ss.SSS'-0400'"))) - UNIX_TIMESTAMP(MIN(to_timestamp(t.event_timestamp, "yyyy-MM-dd'T'HH:mm:ss.SSS'-0400'")))) AS diff_max_min_time_stamp_sec,
  m.category_id,
  t.event_info,
  m.title,
  ROUND(AVG(m.price),2) AS avg_price,
  t.item_bought  
FROM global_temp.train t
LEFT JOIN global_temp.metadata m ON m.item_id = t.event_info
GROUP BY t.id, t.event_info, m.title, m.category_id, t.item_bought
SORT BY t.id ASC

""")

In [29]:
train_query.show(5)

+---+--------------+--------+----------+--------------------+--------------------+---------------------------+-----------+-------------+--------------------+---------+-----------+
| id|unq_event_info|sum_view|sum_search|     first_timestamp|      last_timestamp|diff_max_min_time_stamp_sec|category_id|   event_info|               title|avg_price|item_bought|
+---+--------------+--------+----------+--------------------+--------------------+---------------------------+-----------+-------------+--------------------+---------+-----------+
| 10|             1|       0|         4|2019-10-26 11:59:...|2019-10-26 11:59:...|                         50|       null|HARMAN KARDON|                null|     null|     392483|
| 11|             1|       1|         0|2019-10-05 08:42:...|2019-10-05 08:42:...|                          0|  MLB264139|       151327|Power Bank Pineng...|     49.9|     107670|
| 29|             1|       1|         0|2019-10-16 21:58:...|2019-10-16 21:58:...|                  

In [30]:
test_query = spark.sql("""

SELECT 
  id,
  COUNT(DISTINCT event_info) AS unq_event_info,
  SUM(CASE WHEN event_type = 'view' then 1 else 0 end) AS sum_view,
  SUM(CASE WHEN event_type = 'search' then 1 else 0 end) AS sum_search,
  MIN(to_timestamp(event_timestamp, "yyyy-MM-dd'T'HH:mm:ss.SSS'-0400'")) AS first_timestamp,
  MAX(to_timestamp(event_timestamp, "yyyy-MM-dd'T'HH:mm:ss.SSS'-0400'")) AS last_timestamp,
  (UNIX_TIMESTAMP(MAX(to_timestamp(event_timestamp, "yyyy-MM-dd'T'HH:mm:ss.SSS'-0400'"))) - UNIX_TIMESTAMP(MIN(to_timestamp(event_timestamp, "yyyy-MM-dd'T'HH:mm:ss.SSS'-0400'")))) AS diff_max_min_time_stamp_sec
FROM global_temp.test
GROUP BY id
SORT BY sum_view DESC

""")

In [31]:
test_query.show(5)

+-----------+--------------+--------+----------+--------------------+--------------------+---------------------------+
|         id|unq_event_info|sum_view|sum_search|     first_timestamp|      last_timestamp|diff_max_min_time_stamp_sec|
+-----------+--------------+--------+----------+--------------------+--------------------+---------------------------+
|      15237|           140|     217|       241|2019-10-10 06:52:...|2019-10-16 19:24:...|                     563528|
|25769806291|           122|     183|        84|2019-09-30 14:01:...|2019-10-06 21:43:...|                     546130|
| 8589947960|            71|     124|        67|2019-10-24 20:14:...|2019-10-30 11:53:...|                     488349|
| 8589966965|            64|     122|        50|2019-10-02 18:24:...|2019-10-05 13:17:...|                     240813|
| 8589955165|            62|     115|       113|2019-09-28 06:15:...|2019-10-04 09:55:...|                     531645|
+-----------+--------------+--------+----------+